In [1]:
# ==============================================================================
# CELL 1: SETUP AND IMPORTS
# ==============================================================================
import os
import shutil
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time # <-- ADD THIS

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# --- ADD THESE IMPORTS for ROC/AUC ---
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy.special import softmax
# ------------------------------------

import kagglehub

print("All libraries imported successfully.")

All libraries imported successfully.


In [3]:
# ==============================================================================
# CELL 2: DOWNLOAD AND PREPARE DATASET
# ==============================================================================
# --- Download from Kaggle ---
import kagglehub
path = kagglehub.dataset_download("mdriyadhossain/ai-medleafx")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ai-medleafx


In [4]:
dataset_path = "/kaggle/input/ai-medleafx"
print("Verified path to dataset files:", dataset_path)

# Assuming the structure is dataset_path -> 'Resized Image' -> Plant Type -> Disease/Health Status -> Images
resized_image_base_path = os.path.join(dataset_path, 'Resized Image')

all_class_dirs = []
class_counts = {}

print("\nExploring dataset structure and counting images:")

try:
    # List plant type directories within 'Resized Image'
    plant_type_dirs = [d for d in os.listdir(resized_image_base_path) if os.path.isdir(os.path.join(resized_image_base_path, d))]
    print(f"\nFound {len(plant_type_dirs)} plant types:", plant_type_dirs)

    if not plant_type_dirs:
        print("No plant type directories found within 'Resized Image'. Cannot proceed with expected structure.")
    else:
        # Iterate through each plant type directory
        for plant_type_dir in plant_type_dirs:
            plant_type_path = os.path.join(resized_image_base_path, plant_type_dir)

            # List disease/health status directories within each plant type
            status_dirs = [d for d in os.listdir(plant_type_path) if os.path.isdir(os.path.join(plant_type_path, d))]
            print(f"\nExploring '{plant_type_dir}': Found {len(status_dirs)} status categories:", status_dirs)

            if not status_dirs:
                print(f"No status directories found within '{plant_type_dir}'.")
            else:
                # Iterate through each status directory (these are the actual classes)
                for status_dir in status_dirs:
                    class_name = f"{plant_type_dir}_{status_dir}" # Combine plant type and status for a unique class name
                    class_path = os.path.join(plant_type_path, status_dir)
                    all_class_dirs.append(class_name)

                    # Count image files in the class directory
                    image_files_in_class = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg','.JPG'))]
                    class_counts[class_name] = len(image_files_in_class)
                    print(f"- {class_name}: {class_counts[class_name]} images")

        # Determine the total number of classes
        num_classes = len(all_class_dirs)
        print(f"\nTotal number of classes: {num_classes}")

        # Display class distribution summary
        print("\nClass distribution:")
        for class_name, count in class_counts.items():
            print(f"- {class_name}: {count}")

        # Calculate total number of images
        total_images = sum(class_counts.values())
        print(f"\nTotal number of images: {total_images}")

except FileNotFoundError:
    print(f"Error: The directory '{resized_image_base_path}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


print("\nDataset exploration and counting complete.")

Verified path to dataset files: /kaggle/input/ai-medleafx

Exploring dataset structure and counting images:

Found 4 plant types: ['Sojina', 'Neem', 'Camphor', 'HariTaki']

Exploring 'Sojina': Found 3 status categories: ['Yellow Leaf', 'Healthy Leaf', 'Bacterial Spot']
- Sojina_Yellow Leaf: 814 images
- Sojina_Healthy Leaf: 860 images
- Sojina_Bacterial Spot: 804 images

Exploring 'Neem': Found 4 status categories: ['Powdery Mildew', 'Yellow Leaf', 'Healthy Leaf', 'Shot Hole Leaf']
- Neem_Powdery Mildew: 854 images
- Neem_Yellow Leaf: 854 images
- Neem_Healthy Leaf: 1021 images
- Neem_Shot Hole Leaf: 834 images

Exploring 'Camphor': Found 3 status categories: ['Healthy Leaf', 'Shot Hole', 'Bacterial Spot']
- Camphor_Healthy Leaf: 800 images
- Camphor_Shot Hole: 795 images
- Camphor_Bacterial Spot: 801 images

Exploring 'HariTaki': Found 3 status categories: ['Healthy Leaf', 'Shot Hole', 'Bacterial Spot']
- HariTaki_Healthy Leaf: 816 images
- HariTaki_Shot Hole: 802 images
- HariTaki_Ba

In [8]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [9]:
# ==============================================================================
# CELL 3: DATA SPLITTING
# ==============================================================================
import os, shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Define paths
original_base_dir = os.path.join(dataset_path, 'Resized Image')
new_base_dir = '/content/data'

# Collect image paths and labels
data = []
for plant in os.listdir(original_base_dir):
    for status in os.listdir(os.path.join(original_base_dir, plant)):
        label = f"{plant}_{status}"
        path = os.path.join(original_base_dir, plant, status)
        for fname in os.listdir(path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png','.JPG')):
                data.append({'path': os.path.join(path, fname), 'label': label})

df = pd.DataFrame(data)

# Split dataset: 80% train, 10% val, 10% test
train_val, test = train_test_split(df, test_size=0.4, stratify=df['label'], random_state=42)
train, val = train_test_split(train_val, test_size=0.1, stratify=train_val['label'], random_state=42)
# Output summary
total = len(df)
print(f"\n Dataset Split Summary:")
print(f"- Total images: {total}")
print(f"- Training set : {len(train):5d} images ({len(train)/total:.1%})")
print(f"- Validation set: {len(val):5d} images ({len(val)/total:.1%})")
print(f"- Test set     : {len(test):5d} images ({len(test)/total:.1%})\n")

# Copy files
for split_df, split in zip([train, val, test], ['train', 'val', 'test']):
    print(f" Copying {split} files...")
    for _, row in split_df.iterrows():
        dst_dir = os.path.join(new_base_dir, split, row['label'])
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(row['path'], os.path.join(dst_dir, os.path.basename(row['path'])))
    print(f" {split.capitalize()} files copied: {len(split_df)}")

print("\n All files have been split and copied successfully.")


 Dataset Split Summary:
- Total images: 10858
- Training set :  5862 images (54.0%)
- Validation set:   652 images (6.0%)
- Test set     :  4344 images (40.0%)

 Copying train files...
 Train files copied: 5862
 Copying val files...
 Val files copied: 652
 Copying test files...
 Test files copied: 4344

 All files have been split and copied successfully.


In [10]:
IMG_SIZE = 224

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomApply([transforms.RandomRotation(degrees=10)], p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1)], p=0.3),
    transforms.ToTensor(),
    transforms.Lambda(lambda t: t.expand(3, -1, -1) if t.shape[0] == 1 else t),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Lambda(lambda t: t.expand(3, -1, -1) if t.shape[0] == 1 else t),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


#  Create ImageFolder Datasets
data_dir = new_base_dir
train_ds = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=train_tfms)
val_ds   = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=val_tfms)
test_ds  = datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=val_tfms)

# STEP 7: Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

# STEP 8: Final Checks
print("\n DataLoaders are ready.")
print(f"- Train classes : {train_ds.classes}")
print(f"- Num classes    : {len(train_ds.classes)}")
print(f"- Example image size: {IMG_SIZE}x{IMG_SIZE}")
print(" You can now start training!")


 DataLoaders are ready.
- Train classes : ['Camphor_Bacterial Spot', 'Camphor_Healthy Leaf', 'Camphor_Shot Hole', 'HariTaki_Bacterial Spot', 'HariTaki_Healthy Leaf', 'HariTaki_Shot Hole', 'Neem_Healthy Leaf', 'Neem_Powdery Mildew', 'Neem_Shot Hole Leaf', 'Neem_Yellow Leaf', 'Sojina_Bacterial Spot', 'Sojina_Healthy Leaf', 'Sojina_Yellow Leaf']
- Num classes    : 13
- Example image size: 224x224
 You can now start training!


In [11]:
# ==============================================================================
# CELL 5: DEFINE THE DENSENET201 MODEL
# ==============================================================================
class DenseNet201(nn.Module):
    def __init__(self, num_classes, pretrained=True, freeze_backbone=True):
        super().__init__()
        
        # 1. Load the DenseNet201 model and weights
        weights = models.DenseNet201_Weights.DEFAULT if pretrained else None
        self.densenet = models.densenet201(weights=weights)

        if freeze_backbone:
            # Freeze all parameters in the DenseNet backbone
            for p in self.densenet.parameters():
                p.requires_grad = False

        # 2. Modify the final fully connected layer (DenseNet uses 'classifier')
        # Get the input features of the existing classification layer
        in_features = self.densenet.classifier.in_features
        # Replace the layer with a new one for your specific number of classes
        self.densenet.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # Pass the input through the DenseNet model
        return self.densenet(x)

print("DenseNet201 model class defined.")

DenseNet201 model class defined.


In [12]:
# ==============================================================================
# CELL 6: TRAINING AND EVALUATION LOOP
# ==============================================================================
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import time # Ensure this is imported in Cell 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# *** CHANGE 1: Instantiate DenseNet201 instead of ResNet50 ***
model = DenseNet201(num_classes=len(train_ds.classes), pretrained=False, freeze_backbone=False).to(device)

EPOCHS = 50
best_acc, best_path = 0.0, "medical_plant_best.pth"
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# *** CHANGE 2: Update optimizer params to reference DenseNet's backbone (densenet) ***
optimizer = optim.AdamW([
    {'params': model.densenet.parameters(), 'lr': 1e-5}, 
], weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

# --- Training and Evaluation Functions (These remain the same) ---
def train_one_epoch():
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for x,y in train_loader:
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*x.size(0)
        pred = logits.argmax(1)
        correct += (pred==y).sum().item()
        total += x.size(0)
    return running_loss/total, correct/total

@torch.no_grad()
def evaluate(loader):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        running_loss += loss.item()*x.size(0)
        pred = logits.argmax(1)
        correct += (pred==y).sum().item()
        total += x.size(0)
    return running_loss/total, correct/total

# --- Main Training Loop (This remains the same) ---

# <<<--- START TRAINING TIMER ---<<<
training_start_time = time.time()

print(f"Starting training for {EPOCHS} epochs on {device}...")
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = train_one_epoch()
    va_loss, va_acc = evaluate(val_loader)
    scheduler.step()

    # Log results
    train_losses.append(tr_loss)
    train_accuracies.append(tr_acc)
    val_losses.append(va_loss)
    val_accuracies.append(va_acc)

    # Save the best model
    if va_acc > best_acc:
        best_acc = va_acc
        torch.save(model.state_dict(), best_path)
        print(f"✓ Saved new best model to {best_path} (val acc {best_acc:.4f})")

    print(f"Epoch {epoch:02d} | Train Loss {tr_loss:.4f} Acc {tr_acc:.4f} | Val Loss {va_loss:.4f} Acc {va_acc:.4f}")

# >>>--- END TRAINING TIMER --->>>
training_end_time = time.time()
training_duration = training_end_time - training_start_time

print(f"\nFinished Training. Best Validation Accuracy: {best_acc:.4f}")
print(f"Total Training Time: {training_duration:.2f} seconds ({training_duration/60:.2f} minutes)")

# --- Final Test Evaluation ---

# Load the best model weights
model.load_state_dict(torch.load(best_path))

test_start_time = time.time()
test_loss, test_acc = evaluate(test_loader)
test_end_time = time.time()
test_duration = test_end_time - test_start_time

print(f"\nPerforming final evaluation on the unseen test set...")
print(f"FINAL TEST RESULTS -> Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")
print(f"Testing Time: {test_duration:.2f} seconds")

Starting training for 50 epochs on cuda...
✓ Saved new best model to medical_plant_best.pth (val acc 0.4540)
Epoch 01 | Train Loss 2.2378 Acc 0.3170 | Val Loss 1.8833 Acc 0.4540
✓ Saved new best model to medical_plant_best.pth (val acc 0.5706)
Epoch 02 | Train Loss 1.8163 Acc 0.4879 | Val Loss 1.5662 Acc 0.5706
✓ Saved new best model to medical_plant_best.pth (val acc 0.6488)
Epoch 03 | Train Loss 1.5708 Acc 0.5751 | Val Loss 1.3509 Acc 0.6488
✓ Saved new best model to medical_plant_best.pth (val acc 0.7285)
Epoch 04 | Train Loss 1.3750 Acc 0.6546 | Val Loss 1.1924 Acc 0.7285
✓ Saved new best model to medical_plant_best.pth (val acc 0.7853)
Epoch 05 | Train Loss 1.2359 Acc 0.7223 | Val Loss 1.1003 Acc 0.7853
✓ Saved new best model to medical_plant_best.pth (val acc 0.8021)
Epoch 06 | Train Loss 1.1529 Acc 0.7579 | Val Loss 1.0375 Acc 0.8021
✓ Saved new best model to medical_plant_best.pth (val acc 0.8206)
Epoch 07 | Train Loss 1.0828 Acc 0.7863 | Val Loss 0.9657 Acc 0.8206
✓ Saved new 